## <center>Loading Data for Analysis in other Notebooks</center>

MAKE SURE that all analysis parameters are actually presented in theis notebook. There are tedius ones found in other notebooks (hardcoded) where the calculations are actually performed, but now we have multiple points of analysis calc that hook into this dict to get the parameters for different data, NEED all pars in 1 place to avoid mistakes

REPLACE TICA_INPUTS workflow field
is not that, it is raw data.
Feature data, name / organize as such

#### 1. imports for analyzing trajectory data
-----------------------------------------

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Builtins
import traceback
from pprint import pprint, pformat
from operator import itemgetter
from itertools import chain
from functools import reduce
from pathlib import Path
from multiprocessing import Pool

# Who needs these!?
import warnings
warnings.filterwarnings('ignore')

# Installed
import colorama
import pandas
import seaborn
import colorsys
from scipy.stats import pearsonr
from IPython.display import display, clear_output

# 2lazy 2change names throughout notebooks
import aswanalysis.moretools as aswa_tools
import aswanalysis.workflow as worflow

# TODO use a helper function to
# export common packages to top
# namespace and keep rest of aswa
# bound to parent module name
import aswanalysis as aswa
matplotlib = aswa.plots.matplotlib
cm         = aswa.plots.matplotlib.cm
plt        = aswa.plots.plt
mdtraj     = aswa.mdtraj
pyemma     = aswa.pyemma
coor       = aswa.coor
np         = aswa.np
msm        = aswa.msm
subprocess = aswa.subprocess
shlex      = aswa.shlex

# THE WORKFLOW DATA
from datasetup import *

print(colorama.Back.CYAN)
print("Notebook-level printing will follow a scheme")
print(
    "with " + \
    colorama.Back.GREEN   + "messages"  + colorama.Back.RESET + ", " + \
    colorama.Back.MAGENTA + "data info" + colorama.Back.RESET + ", " + \
    colorama.Back.RED     + "errors"    + colorama.Back.RESET + ", " + \
    colorama.Back.YELLOW  + "warnings"  + colorama.Back.RESET
)
print(colorama.Fore.BLUE + "MAYBE other colorings will be used by notebook also")
print("but plain text messages should arise from packages the")
print("notebook is running. Some notebook messages will guide you")
print("to understanding if problems are happening that violate")
print("expectations of the datasets but aren't actually errors." + colorama.Fore.RESET)

donestamp = lambda: print(colorama.Back.LIGHTBLACK_EX + \
      colorama.Fore.BLUE + \
      "  DONE LOADING DATA  ")


Notebook-level printing will follow a scheme
with messages, data info, errors, warnings
MAYBE other colorings will be used by notebook also
but plain text messages should arise from packages the
notebook is running. Some notebook messages will guide you
to understanding if problems are happening that violate
expectations of the datasets but aren't actually errors.


#### 2. File & Dataset Organization for saving and loading after calculation
--------------------------------------

In [2]:
analysis_directory = Path("analyses")

_model_name = lambda feat, kwargs: "__".join([feat] + [
    "%s-%s"%(str(k),str(v)) for k,v in kwargs.items()])

normalize_cols = lambda arr: np.real(arr) / np.linalg.norm(np.real(arr), axis=0)

def correctone(setups, key, val):
    return next(filter(
        lambda x: x["kwargs"][key] == val,
        setups,
    ))

#### 3. Load multiple datasets into structure organizing the workflows
-------------------------------------

Datasets will be "lined up" for epoch-to-epoch comparison even if different timestep, n replicates, etc.

----------------------------------------

In [3]:
print(colorama.Back.GREEN+"Processing these workflow datasets")
print(colorama.Back.GREEN+"that were defined in file: datasetup.py")
print(colorama.Back.MAGENTA+", ".join(["%s"%wnm for wnm in workflows]))

# filterkey prevents loading unwanted files
#   - the data will be trimmed down and marked with tag "stride"
#     if already happened, we don't include in list of source data
aswa.create_trajlist(workflows, filterkey=lambda fnm: "stride" not in fnm)
aswa.assign_stride(workflows)
aswa.determine_epochsize(workflows)
aswa.determine_datashape(workflows, atomselection=heavies)

Processing these workflow datasets
that were defined in file: datasetup.py
ltj, xma_Ca_3, umi_Ca_3


#### 4. Create Feature Definitions
--------------

In [4]:
feat_CB  = "invca_ba"
feat_Ca  = "invca"
features = feat_Ca,# feat_CB

feature_selections = {
    # Corresponding PyEMMA API:
    #
    #   featurizer.add_inverse_distances(
    #       featurizer.select_Ca()
    #   )
    #
    feat_Ca : {
        "add_inverse_distances":
            {"select_Ca": None}},
    # Corresponding PyEMMA API:
    #  - A list results in series of calls to add
    #    multiple types of features
    #
    #    featurizer.add_inverse_distances(
    #        featurizer.select_Ca()
    #    )
    #    featurizer.add_sidechain_torsions(
    #        which=["chi1"]
    #    )
    #    featurizer.add_backbone_torsions()
    #
 #   feat_CB : [
  #      {"add_inverse_distances": {"select_Ca": None}},
   #     #{"add_chi1_torsions": None},
    #    {"add_sidechain_torsions": None,
     #    "kwargs": {"which": ["chi1"]}},
      #  {"add_backbone_torsions": None},]
}

#### 5. Copy datasets to prepare for different analyses
----------------------------------

In [5]:
all_models = dict({kk:
    dict({
        k : aswa_tools.copy_include(v)
        for k,v in workflows.items()
    }) for kk in feature_selections
})

print(colorama.Back.MAGENTA + "Here is your data:" + colorama.Back.RESET)
print(colorama.Fore.BLUE + "Different Feature Selections to apply to trajectories")
print(colorama.Fore.MAGENTA + pformat(feature_selections))
print(colorama.Fore.BLUE + "Pre-processed dataset descriptions" + colorama.Fore.RESET)

for feat,wnm,w in aswa_tools.iter_models(all_models):
    da = analysis_directory / wnm
    if not da.is_dir():
        print(
    colorama.Back.GREEN+"Creating analysis directory")
        da.mkdir()

for nm, dataset in all_models[
    list(feature_selections)[-1]
].items():
    print(colorama.Back.BLUE + nm + colorama.Back.RESET)
    print(colorama.Fore.MAGENTA + pformat(dataset) + colorama.Fore.RESET)

for feat, featz in feature_selections.items():
    for nm, dataset in all_models[feat].items():
        dataset["data_reader"] = aswa.prepare_tica_inputs(
            dataset, topologies[heavies], features=featz)

Here is your data:
Different Feature Selections to apply to trajectories
{'invca': {'add_inverse_distances': {'select_Ca': None}}}
Pre-processed dataset descriptions
ltj
{'datashape': [[60000],
               [265000],
               [170000],
               [125000],
               [125000],
               [125000],
               [120000],
               [255000],
               [250000],
               [180000],
               [315000],
               [450000],
               [250000],
               [385000],
               [135000],
               [280000],
               [250000],
               [250000],
               [250000],
               [250000],
               [250000],
               [250000],
               [250000],
               [250000],
               [250000],
               [250000],
               [250000],
               [250000]],
 'directory': './',
 'epochsize': 24,
 'filename': 'data/traj/longtraj/protein.*-*.dcd',
 'n_trajs': 1,
 'selection': 'protein',
 

#### 6. Define the calculations we will do
------------------------------

1. TICA starts us off. We will look at how well feature space variance is captured at different lags.
2. Not yet used... VAMP tells us which feature is the best for a given lag.
3. Choose a TICA model that provides reduced coordinates.

In [6]:
# PARAMETERS TO SWEEP
print(colorama.Back.CYAN + "This parameter block was set by running through")
print("the notebook calculations, going back and adjusting")
print("and iterating further to test around good values")
print()
print("the workflow names are: {}".format(list(workflows)))

reversible = False
#tica_lags  = [1, 3, 10, 25, 50, 100, 250, 500, 1000, 2500]
tica_lags  = [100, 250, 500, 1000, 2500]
#tica_lags  = [250, 500, 1000]
msm_lags   = tica_lags
hmm_lags   = [2, 5, 10, 50, 250, 1000]
n_clusters = [75, 300, 600]  # for k-means clustering
#d_clusters = [1.3, 2.0, 2.75, 3.5, 5]     # for regspace clustering
#d_clusters = [2.25, 3.5]
#d_clusters = [3.0, 3.75, 4.5]
#d_clusters = [4.5]
d_clusters = []
#n_macrostates = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
#n_macrostates = [2, 3, 4, 6, 8]
n_macrostates = [2, 3, 4]
n_tica_dim    = 5

print(colorama.Back.RESET + colorama.Fore.LIGHTBLUE_EX + "tica_lags    : " + colorama.Back.LIGHTBLUE_EX + colorama.Fore.WHITE + pformat(tica_lags))
print(colorama.Back.RESET + colorama.Fore.LIGHTBLUE_EX + "n_clusters   : " + colorama.Back.LIGHTBLUE_EX + colorama.Fore.WHITE + pformat(n_clusters))
print(colorama.Back.RESET + colorama.Fore.LIGHTBLUE_EX + "d_clusters   : " + colorama.Back.LIGHTBLUE_EX + colorama.Fore.WHITE + pformat(d_clusters))
print(colorama.Back.RESET + colorama.Fore.LIGHTBLUE_EX + "n_macrostates: " + colorama.Back.LIGHTBLUE_EX + colorama.Fore.WHITE + pformat(n_macrostates))

# Control which datasets we actually analyze
included   = lambda nm: nm in workflows

# THESE: Guess First
#       Update Later
chosen_tica_lag = 1000 # from tica, collective coords for clustering
chosen_tau = 1000 # from its, lag for MSM
chosen_k   = 300 # from k-means, n states for MSM
chosen_d   = 3.75 # from reg-space, n states for MSM
chosen_cl  = "cluster_kmeans"

This parameter block was set by running through
the notebook calculations, going back and adjusting
and iterating further to test around good values

the workflow names are: ['ltj', 'xma_Ca_3', 'umi_Ca_3']
tica_lags    : [100, 250, 500, 1000, 2500]
n_clusters   : [75, 300, 600]
d_clusters   : []
n_macrostates: [2, 3, 4]


In [7]:
# Each calculation sweep is set up here
# TICA
calc_name = "tica"
parameter = dict(lag=tica_lags, reversible=reversible)
wkf_pars  = lambda nm: {"weights": "koopman"} if nm not in {"ltj"} else dict()
fixed     = dict(dim=20) # more than will be used to get good CumVar plot
input_source = "data_reader"
input_key = None
input_val = None
aswa_tools.scan_single_parameter(calc_name, all_models, parameter, fixed, wkf_pars, included, input_source, input_key, input_val)

# VAMP
calc_name = "vamp"
parameter = dict(lag=tica_lags)
wkf_pars  = lambda x: dict()
fixed     = dict(dim=15) # more than will be used to get good CumVar plot
input_source = "data_reader"
input_key = None
input_val = None
aswa_tools.scan_single_parameter(calc_name, all_models, parameter, fixed, wkf_pars, included, input_source, input_key, input_val)

# CLUSTER_KMEANS
calc_name = "cluster_kmeans"
parameter = dict(k=n_clusters)
fixed     = dict(max_iter=100)
wkf_pars  = lambda nm: {"stride": 5} if nm else dict()
input_source = "tica"
input_key = "lag"
input_val = chosen_tica_lag
aswa_tools.scan_single_parameter(
    calc_name, all_models, parameter, fixed,
    wkf_pars, included, input_source, input_key,
    input_val)

# CLUSTER_REGSPACE
calc_name = "cluster_regspace"
# This is VERY dependent on data,
# calculated right before clustering
parameter = dict(dmin=d_clusters)
fixed     = dict(max_iter=100, max_centers=1000, min_counts=[3, 10])
wkf_pars  = lambda nm: {"stride": 5} if nm else dict()
input_source = "tica"
input_key = "lag"
input_val = chosen_tica_lag
aswa_tools.scan_single_parameter(calc_name, all_models, parameter, fixed, wkf_pars, included, input_source, input_key, input_val)

# TODO MSM parameters here

print(colorama.Back.BLUE+"Parameter binding complete")

Parameter binding complete


#### 7. More organization and labels for plots
----------------------------

In [8]:
# RC for plots
seaborn.set(font_scale=1.7)
seaborn.set_style("ticks", {
    "font.family" : ["serif"],
    #"font.scale"  : "1.7",
    "font.serif"  : ["Times New Roman"]
})

cmap = cm.get_cmap("cool")

#cc2 = seaborn.cubehelix_palette(14, start=9, rot=8, dark=0.5, light=.95)
from collections import deque
def get_hls_palette(N):
    cc2 = deque(seaborn.hls_palette(N, l=0.6, s=0.5))
    cc2.rotate(-1) # red is first and confusing the appearance, make last
    return cc2

rmsd_analysis_directory = analysis_directory / "rmsd-simple"
if not analysis_directory.is_dir():
    analysis_directory.mkdir()

if not rmsd_analysis_directory.is_dir():
    rmsd_analysis_directory.mkdir()

label_heavies   = "Heavy RMSD [Å]"
label_alphies   = "Cα RMSD [Å]"
label_timesteps = "Time [μs]"
label_longtraj  = "Single Trajectory"

# Human-read from OpenMM `StateDataReporter` output
md_state_properties  = [
    "Step", "Potential Energy (kJ/mole)",
    "Kinetic Energy (kJ/mole)", "Total Energy (kJ/mole)",
    "Temperature (K)", "Box Volume (nm^3)",
    "Density (g/mL)", "Speed (ns/day)"
]
timestep, e_potential, e_kinetic, e_total, \
temperature, volume, density, md_speed = md_state_properties

md_state_types = _type  = {
    k:v for k,v in zip(md_state_properties,
        [int, float, float, float,
        float, float, float, float]
)}

# Things to help us later on
# NOTE ns_per_step came from file `datasetup.py`
step_per_ns = {label_longtraj:int(1/ns_per_step)}
us_per_step = ns_per_step / 1000
top_file    = reference[heavies]
n_features  = len(features)

_steps_to_ns = lambda s: s * ns_per_step
_index_to_time = lambda idx: idx/step_per_ns[label_longtraj]/1000

match_keys = lambda d,s: [
    key for key in d if key.find(s) > -1]

-------------

- Custom function to match 'test' data with master data based on tica lag time

In [9]:
def get_the_right_master(lag, setups):
    if 0 < lag <= 375:
        mlag = 250
    elif 375 < lag <= 750:
        mlag = 500
    elif 750 < lag <= 1750:
        mlag = 1000
    elif 1750 < lag:
        mlag = 2500
    for setup in setups:
        #print(setup["result"].lag)
        if setup["result"].lag == mlag:
            return {"result": setup["result"]}

- Custom function to create rolling RMSD calculation

In [10]:
def rolling_rmsd(traj, lag=1):#, skip=0):
    assert isinstance(lag, int)
    #assert isinstance(skip, int)
    assert isinstance(traj, mdtraj.Trajectory)
    
    if isinstance(traj._rmsd_traces, type(None)):
        traj.center_coordinates()
    
    def this_rmsd(i): return mdtraj.rmsd(
        traj[i-1], traj[i]#, precentered=True
    )
    
    # ERROR cant pickle
    # TODO  check out pathos.multiprocessing
    #       which uses dill instead
#    pool = Pool(len(traj)-1)
    rrmsds = np.concatenate(
  #      pool.map(this_rmsd, range(1, len(traj)))
        [this_rmsd(i) for i in range(1, len(traj))]
    )
    
    return rrmsds

-------------

#### 8.traj_heaviesrays of Trajectory Energy Data from logfiles
---------------------------------------

In [11]:
# Read Energy from Log Files
traj_energies  = aswa_tools.get_state_trajectories(Path("data/traj/longtraj/logs/"))
traj_energy_df = pandas.DataFrame({
    e_total        : traj_energies[e_total],
    e_potential    : traj_energies[e_potential],
    e_kinetic      : traj_energies[e_kinetic],
    label_timesteps: [5 * i * us_per_step for i in range(len(traj_energies[timestep]))]
})

#### 9. Load or Calc&Save Arrays of Trajectory RMSD Data from trajfiles
--------------------------

In [12]:
for feat,nm,dataset in aswa_tools.iter_models(all_models):

    dataset["rmsd_trajs"] = rmsd_trajs = list()

    for ftj in map(lambda f: Path(f), dataset["tica_inputs"]):

        if not ftj.is_file():
            print(colorama.Back.RED+\
                  "Traj data missing: %s"%dwtj)
            continue

        ftj_rmsd = Path("%s-rmsd-%s.npz" % (
            str(ftj).rstrip(".dcd"), feat))

        if ftj_rmsd.is_file():
            print(colorama.Fore.MAGENTA+\
                  "Found RMSDs File: %s"%ftj_rmsd)
            
            _rtj = np.squeeze(np.load(str(ftj_rmsd))["rmsd"])
            
        else:
            print(colorama.Fore.GREEN+\
                "Calculating and Saving to File: {}".format(
                ftj_rmsd))
            
            _rtj = aswa.calc_rmsd_trajs(
                str(ftj),
                topfile=str(reference[heavies]),
            )[0]
            
            np.savez(str(ftj_rmsd), rmsd=_rtj)

        rmsd_trajs.append(10*_rtj)

# FIXME
# TODO erase all use of these objects
#      since we want to use general structures
traj_heavies   = mdtraj.load(
    all_models['invca']['ltj']['tica_inputs'],
    top=top_file)

c_alphas       = traj_heavies.topology.select(alphies)
folded_heavies = mdtraj.load(top_file)
folded_alphies = folded_heavies.atom_slice(c_alphas)

traj_alphies = traj_heavies.atom_slice(c_alphas)
traj_rmsd_df = pandas.DataFrame({
    label_heavies   : mdtraj.rmsd(
        traj_heavies, folded_heavies) * 10, #nm to angstrom
    label_alphies   : mdtraj.rmsd(
        traj_alphies, folded_alphies) * 10, #nm to angstrom
    label_timesteps : [
        i * us_per_step for i in range(len(traj_alphies))]
})

Found RMSDs File: data/traj/longtraj/protein.00001-00012--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00013-00065--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00066-00099--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00100-00124--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00125-00149--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00150-00174--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00175-00198--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00199-00249--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00250-00299--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00300-00335--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00336-00398--stride--1-rmsd-invca.npz
Found RMSDs File: data/traj/longtraj/protein.00399-00488--stride--1-rmsd-invca.npz
Foun

<center>The lengths of these should match this way...<\center>

In [13]:
try:
    assert len(traj_energies["Step"])*5 == len(traj_alphies)

except AssertionError:
    print(colorama.Back.YELLOW+"ERROR in processing, number of frames in traj logs and traj data doesn't match!")
    print(colorama.Back.YELLOW+"Logs: %d, Data: %d"%(len(traj_energies["Step"])*5, len(traj_alphies)))
    print(colorama.Back.RESET+colorama.Fore.RED+"Missing some traj or log data, but the analysis should be fine")
    print("if you are sure you have all the correct traj data")

ERROR in processing, number of frames in traj logs and traj data doesn't match!
Logs: 1885000, Data: 6490000
Missing some traj or log data, but the analysis should be fine
if you are sure you have all the correct traj data


#### $\infty$ DONE
--------------------

In [14]:
print(colorama.Back.LIGHTBLACK_EX + \
      colorama.Fore.BLUE + \
      "  DONE LOADING DATA  ")

  DONE LOADING DATA  


-------------
-------------

-----------

-----------

-------------
-------------

-----------

-----------

-------------
-------------

-----------

-----------

-------------
-------------

---------